# Fine-tuning CNNs for Classification_keras

In [41]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage import io

In [42]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50 
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

In [43]:
train_df = pd.read_csv('train.csv')
valid_df = pd.read_csv('test.csv')

## Setting up the image augmentation

In [44]:
IMG_SIZE = (224, 224)

In [45]:
train_idg = ImageDataGenerator(rescale=1. / 255.0,
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=20, 
                              shear_range = 0.1,
                              zoom_range=0.1)

train_gen = train_idg.flow_from_dataframe(dataframe=train_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 9
                                         )

# the validation data should not be augmented
val_idg = ImageDataGenerator(rescale=1. / 255.0
                                 )

val_gen = val_idg.flow_from_dataframe(dataframe=valid_df, 
                                         directory=None, 
                                         x_col = 'img_path',
                                         y_col = 'class',
                                         class_mode = 'binary',
                                         target_size = IMG_SIZE, 
                                         batch_size = 6) ## We've only been provided with 6 validation images

Found 20 validated image filenames belonging to 2 classes.
Found 6 validated image filenames belonging to 2 classes.


In [46]:
## Pull a single large batch of random validation data for testing after each epoch
testX, testY = val_gen.next()

## Load in VGG16 with pre-trained ImageNet weights: 

In [47]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [48]:
transfer_layer = model.get_layer('block5_pool')
vgg_model = Model(inputs=model.input,
                   outputs=transfer_layer.output)

In [49]:
## choose the layers of VGG16 want to fine-tune 
## freeze all but the last convolutional layer
for layer in vgg_model.layers[0:17]:
    layer.trainable = False

In [50]:
for layer in vgg_model.layers:
    print(layer.name, layer.trainable)

input_3 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 True
block5_pool True


In [51]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1, activation='sigmoid'))

In [52]:
## Set our optimizer, loss function, and learning rate
optimizer = Adam(lr=1e-4)
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']

In [53]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [54]:
## run a single epoch to check:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 22s 7s/step - loss: 0.8365 - binary_accuracy: 0.4000 - val_loss: 0.7348 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 10s 3s/step - loss: 0.7439 - binary_accuracy: 0.5000 - val_loss: 0.6652 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 9s 3s/step - loss: 0.6122 - binary_accuracy: 0.6500 - val_loss: 0.6499 - val_binary_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 8s 3s/step - loss: 0.5539 - binary_accuracy: 0.8500 - val_loss: 0.6422 - val_binary_accuracy: 0.6667
Epoch 5/5
3/3 [==============================] - 8s 3s/step - loss: 0.5639 - binary_accuracy: 0.8000 - val_loss: 0.6303 - val_binary_accuracy: 0.6667


## Add a few more dense layers

In [55]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [56]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [57]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 14s 5s/step - loss: 0.9843 - binary_accuracy: 0.2500 - val_loss: 0.6207 - val_binary_accuracy: 0.6667
Epoch 2/5
3/3 [==============================] - 12s 4s/step - loss: 0.9306 - binary_accuracy: 0.4500 - val_loss: 0.9841 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 13s 4s/step - loss: 0.6004 - binary_accuracy: 0.7000 - val_loss: 0.5393 - val_binary_accuracy: 1.0000
Epoch 4/5
3/3 [==============================] - 12s 4s/step - loss: 0.6824 - binary_accuracy: 0.5500 - val_loss: 0.6823 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 12s 4s/step - loss: 0.4577 - binary_accuracy: 0.8000 - val_loss: 0.5603 - val_binary_accuracy: 0.6667


## Add dropout and another fully connected layer:

In [58]:
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(vgg_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(512, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dense(256, activation='relu'))

# Add a dense (aka. fully-connected) layer.
# Change the activation function to sigmoid 
# so output of the last layer is in the range of [0,1] 
new_model.add(Dense(1, activation='sigmoid'))

In [59]:
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [61]:
## Just run a single epoch to see how it does:
new_model.fit_generator(train_gen, 
                                  validation_data = (testX, testY), 
                                  epochs = 5)

Epoch 1/5
3/3 [==============================] - 13s 4s/step - loss: 0.6667 - binary_accuracy: 0.6000 - val_loss: 0.6866 - val_binary_accuracy: 0.5000
Epoch 2/5
3/3 [==============================] - 13s 4s/step - loss: 0.7808 - binary_accuracy: 0.4500 - val_loss: 0.7294 - val_binary_accuracy: 0.5000
Epoch 3/5
3/3 [==============================] - 13s 4s/step - loss: 0.7099 - binary_accuracy: 0.5000 - val_loss: 0.6696 - val_binary_accuracy: 0.6667
Epoch 4/5
3/3 [==============================] - 12s 4s/step - loss: 0.5626 - binary_accuracy: 0.6500 - val_loss: 0.6383 - val_binary_accuracy: 0.5000
Epoch 5/5
3/3 [==============================] - 11s 4s/step - loss: 0.6976 - binary_accuracy: 0.6000 - val_loss: 0.6184 - val_binary_accuracy: 0.6667


 there were the fewest parameters to train since only fine-tuning the last layer of VGG16. 

The last architecture seemed to show more stable than the second. Likely due to the fact that added Dropout. 